# **Face Recognition**

In [9]:
!pip install facenet-pytorch torchvision --upgrade
!pip install pillow matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import transforms
import torch

# **Device Setup**

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_dir = "/content/drive/MyDrive/Comys_Hackathon5/Task_B/train"
val_dir = "/content/drive/MyDrive/Comys_Hackathon5/Task_B/val"

# **FaceNet Model Loading**

In [4]:
facenet = InceptionResnetV1(pretrained='vggface2', classify=False).eval().to(device)
mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device=device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

# **Test Time Augmentation (TTA)**

In [9]:
def tta_embeddings(img):
    variants = [
        lambda x: x,
        lambda x: ImageEnhance.Brightness(x).enhance(1.2),
        lambda x: ImageEnhance.Contrast(x).enhance(1.2),
        lambda x: ImageEnhance.Sharpness(x).enhance(1.5),
        lambda x: x.transpose(Image.FLIP_LEFT_RIGHT),
    ]
    embeddings = []
    for transform in variants:
        augmented = transform(img)
        face = mtcnn(augmented)
        if face is not None:
            with torch.no_grad():
                emb = facenet(face.unsqueeze(0).to(device)).cpu().numpy()
                embeddings.append(emb)
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros((1, 512))

# **Embeddings**

In [10]:
def build_reference_embeddings(train_dir):
    embeddings, labels = [], []
    for person_name in tqdm(os.listdir(train_dir)):
        person_path = os.path.join(train_dir, person_name)
        if not os.path.isdir(person_path):
            continue
        for file in os.listdir(person_path):
            image_path = os.path.join(person_path, file)
            if os.path.isdir(image_path):
                continue  # skip directories
            try:
                img = Image.open(image_path).convert('RGB')
                emb = tta_embeddings(img)
                embeddings.append(emb)
                labels.append(person_name)
            except Exception as e:
                print(f"[ERROR] Skipping file: {image_path} due to {e}")
    return np.vstack(embeddings), labels


# **Inference on Distorted Data**

In [11]:
SIMILARITY_THRESHOLD = 0.6  # can tune based on validation

def evaluate(test_dir, ref_embeddings, ref_labels):
    y_true, y_pred, scores = [], [], []
    for person_folder in os.listdir(test_dir):
        person_path = os.path.join(test_dir, person_folder)
        if not os.path.isdir(person_path):
            continue
        for file in os.listdir(person_path):
            image_path = os.path.join(person_path, file)
            if os.path.isdir(image_path):
                continue  # skip directories
            try:
                img = Image.open(image_path).convert('RGB')
                emb = tta_embeddings(img)
                sim = cosine_similarity(emb.reshape(1, -1), ref_embeddings)[0]
                max_score = np.max(sim)
                pred_label = ref_labels[np.argmax(sim)]
                true_label = person_folder

                y_true.append(1 if pred_label == true_label else 0)
                y_pred.append(1 if max_score > SIMILARITY_THRESHOLD else 0)
                scores.append(max_score)

                if max_score < SIMILARITY_THRESHOLD:
                    print(f"[LOW MATCH] {file} -> {pred_label} (Score: {max_score:.2f})")
            except Exception as e:
                print(f"[ERROR] Skipping file: {image_path} due to {e}")

    print(f"\n✅ Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"✅ F1 Score:  {f1_score(y_true, y_pred):.4f}")
    return y_true, y_pred, scores

# **Example Usage**

In [12]:
if __name__ == "__main__":
    print("🔍 Building reference embeddings...")
    ref_embs, ref_labels = build_reference_embeddings(train_dir)

    print("\n🔬 Evaluating on distorted/test set...")
    evaluate(val_dir, ref_embs, ref_labels)

🔍 Building reference embeddings...


100%|██████████| 877/877 [1:08:24<00:00,  4.68s/it]



🔬 Evaluating on distorted/test set...
[LOW MATCH] Wally_Szczerbiak_0001.jpg -> Kaio_Almeida (Score: 0.51)
[LOW MATCH] William_Delahunt_0001.jpg -> Martin_Bandier (Score: 0.57)
[LOW MATCH] William_Cocksedge_0001.jpg -> Colin_Powell (Score: 0.59)
[LOW MATCH] William_Genego_0001.jpg -> James_Lockhart (Score: 0.54)
[LOW MATCH] Will_Ferrell_0001.jpg -> Larry_Ellison (Score: 0.49)
[LOW MATCH] Yoelbi_Quesada_0001.jpg -> Tono_Suratman (Score: 0.47)
[LOW MATCH] Thomas_Manger_0001.jpg -> Steve_Mariucci (Score: 0.58)
[LOW MATCH] Tomoko_Hagiwara_0002.jpg -> Tyler_Grillo (Score: 0.43)
[LOW MATCH] Tomoko_Hagiwara_0001.jpg -> Wang_Nan (Score: 0.54)
[LOW MATCH] Tom_Koenigs_0001.jpg -> Charlton_Heston (Score: 0.57)
[LOW MATCH] Troy_Hudson_0001.jpg -> James_Mathis (Score: 0.56)
[LOW MATCH] Troy_Aikman_0001.jpg -> Scott_McNealy (Score: 0.57)
[LOW MATCH] Thierry_Mariani_0001.jpg -> 095_frontal (Score: 0.49)
[LOW MATCH] Tim_Floyd_0001.jpg -> Ilan_Ramon (Score: 0.54)
[LOW MATCH] Tim_Floyd_0002.jpg -> Oswal